In [ ]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - CIFAR10

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'cifar10'
dataset_percentage = '5'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for cifar10_5...
Found 1750 images belonging to 10 classes.
Loading test set for cifar10_5...
Found 500 images belonging to 10 classes.
Loading validation set for cifar10_5...
Found 250 images belonging to 10 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
55/55 [==============================] - 1s 25ms/step - loss: 5.2012 - accuracy: 0.1114 - precision: 0.0274 - recall: 0.0219 - val_loss: 2.3174 - val_accuracy: 0.1280 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00

Epoch 00001: val_loss improved from inf to 2.31741, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/cifar10/best.h5
Epoch 2/50
55/55 [==============================] - 1s 12ms/step - loss: 2.2378 - accuracy: 0.2034 - precision: 0.0364 - recall: 0.0106 - val_loss: 2.3620 - val_accuracy: 0.1980 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00

Epoch 00002: val_loss did not improve from 2.31741
Epoch 3/50
55/55 [==============================] - 1s 13ms/step - loss: 1.9850 - accuracy: 0.2891 - precision: 0.1455 - recall: 0.0785 - val_loss: 2.5647 - val_accuracy: 0.2420 - val_precision: 0.1875 - val_recall: 0.0646

Epoch 00003: val_loss did not improve from 2.31741
Epoch 4/50
55/55 [==============================] - 1s 13ms/step 

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,1.4406347274780273,2.317410945892334,0.12800000607967377,0.0,0.0
2,0.6998801231384277,2.3620123863220215,0.1979999989271164,0.0,0.0
3,0.7207968235015869,2.564748764038086,0.24199999868869781,0.1874999850988388,0.06458333134651184
4,0.787477970123291,2.083345413208008,0.23000000417232513,0.125,0.07500000298023224
5,0.8600518703460693,2.081745147705078,0.27799999713897705,0.2291666567325592,0.140625
6,0.8388378620147705,2.6822471618652344,0.29600000381469727,0.3958333134651184,0.12931548058986664
7,0.9091880321502686,1.8069210052490234,0.30799999833106995,0.3020833432674408,0.1927083283662796
8,0.9406189918518066,2.261868715286255,0.31200000643730164,0.4583333432674408,0.3057291507720947
9,0.7336039543151855,2.340954303741455,0.3179999887943268,0.25,0.1145833358168602
10,0.8313519954681396,3.471841812133789,0.3199999928474426,0.234375,0.10803572088479996
11,0.7495748996734619,1.8138427734375,0.3140000104904175,0

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
0.9753000211715698,48.76500105857849,2.141873598098755,0.3199999928474426,0.3437499701976776,0.16875000298023224
